
##FINANCIAL DATA
MODULE 4 | LESSON 4


---


# **COMPARING LEVERAGED STRATEGIES**

|  |  |
|:---|:---|
|**Reading Time** |  30 minutes |
|**Prior Knowledge** |Option payoffs, Option pricing using Python, Option strategies, P&L concepts, Leverage     |
|**Keywords** |Bull Spread    |


---


*In the previous module, we showed how to get option data imported into Python. This data included not only price, but bid and ask levels, volume and open interest, and implied volatility, among other fields. In this lesson, we compare different ways to enact a bullish trade, using different combinations of stocks, options, and leverage.*

## 1. Importing Real-Time Data

In this lesson, let's get some real-time pricing of stocks and options.

In [ ]:
import math

import opstrat as op
import pandas as pd
import requests
import yfinance as yf
from optionprice import Option
from yahoo_fin import options

In [ ]:
# https://analyticsindiamag.com/top-python-libraries-to-get-historical-stock-data-with-code/

# YOU CAN FIND YOUR OWN ALPHAVANTAGE KEY ON YOUR WQU PROFILE SETTINGS PAGE
myApiKey = "960c8de34c8a98412e839a5fbdfe0cd221f1a3c4c8c9d8d325fe57f8075993e906067d2dab584c82d65e03bced836f16a7a13608bd0cea16341dc327e575e7f484bfec871c14ad2efdb4776aa99c2ce8981e7be03467bf5df642de42876df30c32e58bb8084da9be41373c124a846a61aebe01d0321d66aa09dd87a38836e309"

# Endpoint for getting data from AlphaVantage
url = (
    "https://www.alphavantage.co/query?"
    "function=TIME_SERIES_DAILY&"
    "symbol=NFLX&"
    f"apikey={myApiKey}"
)

# Make GET request
response = requests.get(url=url)

# Get stock data from response
response_data = response.json()
stock_data = response_data["Time Series (Daily)"]

# Load stock data into DataFrame
df = pd.DataFrame.from_dict(stock_data, orient="index", dtype=float)

# Rename columns
df.columns = ["open", "high", "low", "close", "volume"]

# plotting the opening and closing value
df[["close"]].plot();

Note: This cell currently results an error due to subscription limitation associated with NFLX endpoint. AlphaVantage often revise API or plans requiring the code modifications after change. NFLX used to be on free plan but moved to premium. You can try for example 'IBM' ticker to test functionality in above code.

We can alternatively use `yfinance` to get NFLX data.

In [ ]:
# Alternatively, we can use yfinance since NFLX close is given in its adjusted form.

ts = yf.download("NFLX")["Adj Close"]

ts.plot();

## 2. Overview of Strategies

Suppose we have \$100,000 to invest in a bullish strategy for Netflix.

1. Buy stock using cash.
2. Buy stock on margin.
3. Buy a call option.
4. Buy a bull spread (buy call at K, sell call at K+d).

How much data do we need?
In all cases, we certainly need to know the current stock price, specifically the ask price.
For the cash stock strategies, all we need to know is the current ask price.
For the financed stock strategy, we also need to know the financing rate.
For the option strategies, we need to know the option chain.

## 3. Stock Strategy

First, we'll begin with the stock strategy.
Let's suppose you buy NFLX at 333, and you plan to sell it at 345.

In [ ]:
nflx0 = 333
nflx1 = 345

In [ ]:
def BuyShares(capital, sharePrice):
    return math.floor(capital / sharePrice)

In [ ]:
myCapital = 100000
numShares = BuyShares(myCapital, nflx0)
numShares

In [ ]:
def CalculatePnL(sellPrice, buyPrice, numShares):
    return round(numShares * (sellPrice - buyPrice), 2)

You can buy 300 shares

In [ ]:
cashStockPnL = CalculatePnL(nflx1, nflx0, numShares)
cashStockPnL

You made \$3,600. Let's write a quick function to compute percent return.

In [ ]:
def CalculatePctReturn(fundsOut, fundsIn):
    return round(fundsOut / fundsIn * 100, 2)

In [ ]:
CalculatePctReturn(cashStockPnL, myCapital)

The stock increased (345-333)/333 = 3.6%
You bought the stock cash.
Your investment earned 3.6%
This is an example of no leverage.
All the other examples involve some form of leverage.
Consequently, they will have higher returns due to that leverage.

## 4. Financed Stock Strategy

Let's write a short function to calculate functions for buying stocks with borrowed funds.

In [ ]:
def FinanceShares(capital, sharePrice, financePct=0.50):
    return BuyShares(capital / financePct, sharePrice)

In [ ]:
# We can finance 50% of our purchase.
numLvgShares = FinanceShares(myCapital, nflx0)
lvgPnL = CalculatePnL(nflx1, nflx0, numLvgShares)
lvgPnL

In [ ]:
CalculatePctReturn(lvgPnL, myCapital)

## 5. Call Option Strategy

For the call option strategy, we have to select a call and an expiration date.
Clearly, there are many choices.
Suppose we think the stock will increase \$10.

It is prudent to purchase a call that is OTM (strike above the current stock, but not more than \$10; otherwise, it will not be ITM.


In [ ]:
nflx_dates = options.get_expiration_dates("nflx")
print(len(nflx_dates))

In [ ]:
try:
    callsNflx = options.get_calls("nflx")
except:  # noQA E722
    # to load data from local file in case of any disruptions
    callsNflx = pd.read_csv("nflx_calls.csv")

list(callsNflx.columns)

Let's choose a call option with a strike of 340.

In [ ]:
myOption = callsNflx[(340 == callsNflx["Strike"])]
opt = myOption["Last Price"]
myOption

In [ ]:
nflxCall1 = Option(
    european=True, kind="call", s0=333, k=340, t=4, sigma=0.573, r=0.0, dv=0
)
nflxOpt1 = round(nflxCall1.getPrice() * 100, 2)
nflxOpt1

In [ ]:
# Number of options we can buy
numCallOptions = BuyShares(myCapital, nflxOpt1)
numCallOptions

We can buy 198 options; that's a lot of options! Now the stock price jumps to 345. Let's revalue the option at this new price.

In [ ]:
nflxCall2 = Option(
    european=True, kind="call", s0=345, k=340, t=4, sigma=0.573, r=0.0, dv=0
)
nflxOpt2 = round(nflxCall2.getPrice() * 100, 2)
nflxOpt2

Let's calculate the P&L.

In [ ]:
optPnL = CalculatePnL(nflxOpt2, nflxOpt1, numCallOptions)
optPnL

Now, calculate the return.

In [ ]:
CalculatePctReturn(optPnL, myCapital)

## 6. Bull Spread Strategy

In this strategy, we buy a call option at 340 and sell one at 350.
Let's price these options.<span style='color: transparent; font-size:1%'>All rights reserved WQU WorldQuant University QQQQ</span>

In [ ]:
nflxCall1 = Option(
    european=True, kind="call", s0=333, k=340, t=4, sigma=0.573, r=0.0, dv=0
)
nflxOpt1 = round(nflxCall1.getPrice() * 100, 2)

nflxCall2 = Option(
    european=True, kind="call", s0=333, k=350, t=4, sigma=0.573, r=0.0, dv=0
)
nflxOpt2 = round(nflxCall2.getPrice() * 100, 2)

print([nflxOpt1, nflxOpt2])
nflxSpr1 = nflxOpt1 - nflxOpt2

Let's use the `opstrat` package to view the payoff of the bull strategy.

In [ ]:
myCall = {"op_type": "c", "strike": 340, "tr_type": "b", "op_pr": 5.0306}
myPut = {"op_type": "c", "strike": 350, "tr_type": "s", "op_pr": 3.4836}
op_list = [myCall, myPut]
op.multi_plotter(spot=333, spot_range=10, op_list=op_list)

For each call option we buy, we need to sell one.
The sale creates \\$233.33 per contract.
The purchase costs \\$503.06 per contract.
Therefore, the net price is \\$269.73.
The bull spread costs a little more than half the amount that the regular call option costs. This is because we are "rebated" by selling a call that is further OTM.

In [ ]:
# Number of spread options
nflxOptB = nflxOpt1 - nflxOpt2
numSpreadOptions = BuyShares(myCapital, nflxOptB)
numSpreadOptions

In [ ]:
nflxCall3 = Option(
    european=True, kind="call", s0=345, k=340, t=4, sigma=0.573, r=0.0, dv=0
)
nflxOpt3 = round(nflxCall3.getPrice() * 100, 2)

nflxCall4 = Option(
    european=True, kind="call", s0=345, k=350, t=4, sigma=0.573, r=0.0, dv=0
)
nflxOpt4 = round(nflxCall4.getPrice() * 100, 2)

print([nflxOpt3, nflxOpt4])
nflxSpr2 = nflxOpt3 - nflxOpt4

In [ ]:
sprPnL = CalculatePnL(nflxSpr2, nflxSpr1, numSpreadOptions)
sprPnL

In [ ]:
CalculatePctReturn(sprPnL, myCapital)

## 7. Leverage

Stock              3.6%  </br>
Financed Stock     7.2%  </br>
Call Option       116.94%  </br>
Bull Spread       80.9%   </br>

Keep in mind that these options expire in four days.
If the price does not move that much in a short time, we stand to lose 100% of the option investment. This is extremely risky.
Of course, we could find options that have more time to expiration, which
would be more expensive to buy but less risky since the extra time allows more volatility to take place.

Nevertheless, the results are evident.
The most leveraged position is the call option: It has the highest return.
Options have built-in leverage because they control 100 shares of stock.

The next most leveraged position is the bull spread: It has a long call option but is hedged by selling another call option. This limits the upside but is cheaper than merely buying the call option.

The next most leveraged position is using the underlying. Borrowing money to buy assets is a different type of leverage; rather than through a multiplier, it is simply through the fact that the invested funds should earn a higher rate of return than the borrowed funds.

The unleveraged position is simply trading the underlying without any financing. Leveraged returns attract institutional money for the potential of greater returns. Looking at data, such as the put call open interest ratio, tells us how the option traders are behaving through market data.

There is much more data than just prices and returns. When you use derivatives, there are extraordinary amounts of data that indicate both sentiment and fear in the market. In future courses, we'll examine these in more detail, especially when we study implied volatility.


## 8. Conclusion

In this lesson, we compared different ways to enact a bullish trade, using different combinations of stocks, options, and leverage. Leverage can work through borrowed funds or through a derivative controlling a multitude of underlying assets. In the next module, we expand our knowledge of Python to areas of securitization and credit.

---
Copyright 2023 WorldQuant University. This
content is licensed solely for personal use. Redistribution or
publication of this material is strictly prohibited.
